# Solving Sudoku Puzzles
## Assignment Preamble
Please ensure you carefully read all of the details and instructions on the assignment page, this section, and the rest of the notebook. If anything is unclear at any time please post on the forum or ask a tutor well in advance of the assignment deadline.

In addition to all of the instructions in the body of the assignment below, you must also follow the following technical instructions for all assignments in this unit. *Failure to do so may result in a grade of zero.*
* [At the bottom of the page](#Submission-Test) is some code which checks you meet the submission requirements. You **must** ensure that this runs correctly before submission.
* Do not modify or delete any of the cells that are marked as test cells, even if they appear to be empty.
* Do not duplicate any cells in the notebook – this can break the marking script. Instead, insert a new cell (e.g. from the menu) and copy across any contents as necessary.

Remember to save and backup your work regularly, and double-check you are submitting the correct version.

This notebook is the primary reference for your submission. You may write code in separate `.py` files but it must be clearly imported into the notebook so that it runs without needing to reference those files, and you must explain clearly what functionality is contained in those files (through comments, markdown cells, etc).

As always, **the work you submit for this assignment must be entirely your own.** Do not copy or work with other students. Do not copy answers that you find online. These assignments are designed to help improve your understanding first and foremost – the process of doing the assignment is part of *learning*. They are also used to assess your ability, and so you must uphold academic integrity. Submitting plagiarised work risks your entire place on your degree.

**You must implement your own work.** You can and are encouraged to use helper libraries like `numpy`, but you must not use 3rd-party libraries or code that solves the assignment for you. *Even if it is properly referenced*, you only get marks for work you have done. If you are unsure, ask on the forum or ask a tutor directly.

**The pass mark for this assignment is 40%.** We expect that students, on average, will be able to produce a submission which gets a mark between 50-70% within the normal workload allocation for the unit, but this will vary depending on individual backgrounds. Please ask for help if you are struggling.

## Getting Started
For this assignment, you will be writing an agent that can solve sudoku puzzles. You should be familiar with sudoku puzzles from the unit material. You are given a 9x9 grid with some fixed values. To solve the puzzle, the objective is to fill the empty cells of the grid such that the numbers 1 to 9 appear exactly once in each row, column, and 3x3 block of the grid. 

Below is a sample puzzle along with its solution. 

<img src="images/sudoku.png" style="width: 50%;"/>

For this assignment you will need to submit:
1. The implementation for an agent which can solve sudoku puzzles – this notebook
 * You can use any algorithm you like, from the unit material or otherwise
 * Your code will be subject to automated testing, from which grades will be assigned based on whether it can solve sudokus of varying difficulty
 * To get a high grade on this assignment, the speed of your code will also be a factor – the quicker the better
 * There are some sample tests included below, make sure your code is compatible with the format of these tests
2. A text file that explains your approach and the decisions you made in your own words – a readme file
 * Submissions that do not include the written section will receive zero marks – **this part is mandatory**
 * You may write your file in plain text (.txt) or [Markdown](https://www.markdownguide.org/basic-syntax/) (.md)
 * To get top marks on this assignment, as well as getting a high grade from your implementation, you must also demonstrate excellent academic presentation in your written section

### Choice of Algorithm
The choice of algorithm to solve sudoku puzzles is up to you. We expect you will use search techniques from the unit, but you could make something up yourself, or do some independent research to find something else. You will need to evaluate and balance the trade-off between how well suited you think the algorithm is and how difficult it is to write, but there is some advice below.

You should also consider what *data structures* you use within your code. Certain algorithms will work much faster with certain data structures, and understanding this is considered part of understanding the algorithm itself. 

I suggest you implement *constraint satisfaction* as it is described in the unit material. A good implementation of a backtracking depth-first search with constraint propagation should be sufficient to get a good grade in the automated tests (roughly 60-70%). There is an example of constraint satisfaction on the eight-queens puzzle, and you could use this as a guide to help you get started.

You could also write a successful agent that uses the other search techniques you have seen in the unit so far: basic search, heuristic search, or local search. You may find these easier to implement, though they may perform less well. 

To get a high grade on this assignment will require a particularly efficient implementation of constraint satisfaction, or something which goes beyond the material we have presented. *This is left unguided and is not factored into the unit workload estimates.*

If you choose to implement more than one algorithm, please feel free to include your code and write about it in part two (readme file), but only the code in this notebook will be used in the automated testing.

## Sample Sudoku Puzzles
To get started, the cell below will load in some sample sudoku puzzles for you so you can see the format. There are sudokus provided of multiple difficulties (easier sudokus typically start with more digits provided). The cell below only loads the easiest, but there is another test cell lower in the notebook which will run your code against all of the provided puzzles.

Each sudoku is a 9x9 NumPy array of integers, where zero represents an empty square. Each difficulty comes with 15 sudokus, so when you load the file, it is stored in a 15x9x9 array.

In [1]:
import numpy as np
import random

# Load sudokus
sudoku = np.load("data/very_easy_puzzle.npy")
print("very_easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudoku.shape}, sudoku[0].shape: {sudoku[0].shape}, sudoku.dtype: {sudoku.dtype}")

# Load solutions for demonstration
solutions = np.load("data/very_easy_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudoku[0], "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

very_easy_puzzle.npy has been loaded into the variable sudoku
sudoku.shape: (15, 9, 9), sudoku[0].shape: (9, 9), sudoku.dtype: int8

First sudoku:
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]] 

Solution of first sudoku:
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]


## Part One
You should write all of your code for solving sudokus below this cell.

You must include a function called `sudoku_solver(sudoku)` which takes one sudoku puzzle (a 9x9 NumPy array) as input, and returns the solved sudoku as another 9x9 NumPy array. This is the function which will be tested. 

In [1]:
import numpy as np
import copy
def sudoku_solver(sudoku):
    """
    Solves a Sudoku puzzle and returns its unique solution.

    Input
        sudoku : 9x9 numpy array
            Empty cells are designated by 0.

    Output
        9x9 numpy array of integers
            It contains the solution, if there is one. If there is no solution, all array entries should be -1.
    """
    # creates a partially solved sudoku grid up to the point where backtracking is needed
    partial_state = PartialSudokuState(sudoku)
    partial_state.manually_solve_sudoku()
    # checks if partially solved sudoku is invalid
    if partial_state.check_is_invalid():
        # returns array of -1s if it is invalid
        partial_state.sudoku[:] = -1
        return partial_state.sudoku
    # sees if sudoku can be solved without backtracking
    elif partial_state.check_if_sudoku_solved():
        partial_state.finish_sudoku()
        return partial_state.sudoku
    else:
        # performs a depth first search with constraint satisfaction
        goal = depth_first_search(partial_state)
        if goal is not None:
            return goal
        else:
            # if the backtracking returns none then it is an invalid sudoku and returns array of -1
            partial_state.sudoku[:] = -1
            return partial_state.sudoku

def depth_first_search(partial_state):
    """
    Performs depth first search with constraint satisfaction on partial state
    
    Input
        Partial state object
        
    Output
        9x9 NumPy array
    """
    index = pick_next_value(partial_state)
    nums = order_values(partial_state, index)
    for num in nums:
        new_state = partial_state.set_value(index, num)
        # checks if sudoku is valid
        if not new_state.check_is_invalid(): # check rules obeyed
            # checks if the sudoku has been solved: if so it returns it
            if new_state.check_if_sudoku_solved():# check all values filled
                return new_state.sudoku
            else:
                # goes deeper into search
                deep_state = depth_first_search(new_state)
                if deep_state is not None and check_if_goal(deep_state):
                    # returns the state once all values are filled in
                    return deep_state
    return None

def check_if_goal(deep_state):
    """
    Checks to see if state produced by search is valid
    
    Input
        9x9 numpy array
        
    Output
        Boolean indicating if goal reached or not
    """
    # makes sure there is one of each number in each row
    for row in deep_state:
        count_arr = np.bincount(row)
        for num in range(1,10):
            if count_arr[num] != 1:
                return False
    # makes sure there is one of each number in each column
    transpose = deep_state.transpose()
    for column in transpose:
        count_arr = np.bincount(column)
        for num in range(1,10):
            if count_arr[num] != 1:
                return False
    return True
    
def pick_next_value(partial_state):
    """
    Chooses square with the smallest number of possible values to guess
    
    Input
        partial state object
        
    Output
        list containing x and y coordinates of the value that will be guessed next
    """
    minList = [-1, -1]
    minValue = 10
    for x in range(9):
        for y in range(9):
            # finds smallest list and assigns it to minValue
            size = len(partial_state.possible_values[x,y])
            if size < minValue and size > 1:
                minValue = size
                # assigns index of smallest list to minList
                minList = [x, y]
    return minList

def order_values(partial_state, index):
    """
    Orders values so starts with number in list that has least occurences in other possible value lists
    
    Input
        partial state object, coordinates of the list to be ordered
        
    Output
        list of numbers in the order they should be guessed in
    """
    numbers = []
    counts = []
    # iterates through all other possible values and counts number of times number appears
    for num in partial_state.possible_values[index[0], index[1]]:
        num_count = 0
        for lst in partial_state.possible_values:
            num_count += sum(x.count(num) for x in lst)
        numbers.append(num)
        counts.append(num_count)
    # sorts list of numbers by count ascending
    counts, numbers = zip(*sorted(zip(counts, numbers)))
    return numbers
      
class PartialSudokuState:
    def __init__(self, sudoku):
        """
        Initialises an instance of a partialsudokustate object
    
        Input
            9x9 sudoku grid
        """
        self.sudoku = sudoku
        self.new_changes = True
        # assigns the initial array of possible values for each square
        self.possible_values = self.assign_initial_possible_values()
                    
    def manually_solve_sudoku(self):
        """
        Manually solves the sudoku using logical techniques
        
        Output
            the sudoku of the current object
        """
        # sees if sudoku is valid
        if self.check_is_invalid():
            self.sudoku[:] = -1
            return self.sudoku           
        # sees if all boxes have one option left
        if self.check_if_sudoku_solved():
            self.finish_sudoku()
            return self.sudoku
        # if boxes have more than one value in them
        else:
            for n in range(3):
                # sees if sudoku has now been solved
                if self.check_if_sudoku_solved():
                    self.finish_sudoku()
                    return self.sudoku
                else:
                    # if it has not been solved then runs more methods to reduce possible values in the sudoku
                    self.strip_multiple_values()
                    self.constrain_values()
             
    def assign_initial_possible_values(self):
        """
        Creates an array of possible values for each square in the sudoku
        
        Output
            9x9 array of lists of possible values
        """
        possible_values = np.zeros((9,9), list)
        for x in range(9):
            for y in range(9):
                # assigns the numbers given
                if self.sudoku[x,y] != 0:
                    possible_values[x,y] = [self.sudoku[x,y]]
                else:
                    # fills in blank squares
                    values = [1,2,3,4,5,6,7,8,9]
                    possible_values[x,y] = self.assign_possible_values(values, x, y)
        return possible_values
    
    def assign_possible_values(self, values, x, y):
        """
        Removes values from the arrays of possible values in accordance to the rules of sudoku
        
        Output
            9x9 array of lists of possible values
        """
        for num in range(1,11):
            # check if that number already exists in the row
            for i in range(9):
                 if self.sudoku[i, y] == num and num in values:
                    if len(values) > 1:
                        values.remove(num)
            # check if that number already exists in the column
            for j in range(9):
                if self.sudoku[x, j] == num and num in values:
                    if len(values) > 1:
                        values.remove(num)
            # check if that number already exists in the 3x3 square
            squareX = x - x % 3
            squareY = y - y % 3
            for p in range(3):
                for q in range(3):
                    if self.sudoku[p+squareX, q+squareY] == num and num in values:
                        if len(values) > 1:
                            values.remove(num)
        return values
    
    def constrain_values(self):
        """
        Uses advanced sudoku solving techniques (unique candidate, naked subset, sole candidate) to reduce possible values
        
        """
        new_changes = True
        while new_changes:
            new_changes = False
            # unique candidate
            for x in range(0,9,3):
                for y in range(0,9,3):
                    squareX = x - x % 3
                    squareY = y - y % 3
                    for num in range(1,10):
                        count = 0
                        for p in range(3):
                            for q in range(3):
                                if num in self.possible_values[p+squareX, q+squareY]:
                                    count += 1
                                    tempX = p+squareX
                                    tempY = q+squareY
                        if count == 1 and len(self.possible_values[tempX, tempY]) > 1:
                            new_changes = True
                            self.possible_values[tempX, tempY] = [num]                  
            # naked subsets
            # check row for naked double subset
            for x in range(9):
                doubles = []
                for value in self.possible_values[x]:
                    count = len(value)
                    if count == 2:
                        doubles.append(value)
                if len(doubles) == 2:
                    if doubles[0] == doubles[1]:
                        a = doubles[0][0]
                        b = doubles[0][1]
                        for y in range(9):
                            if a in self.possible_values[x,y] and self.possible_values[x,y] != doubles[0]:
                                temp = self.possible_values[x,y]
                                temp.remove(a)
                                self.possible_values[x,y] = temp
                            if b in self.possible_values[x,y] and self.possible_values[x,y] != doubles[0]:
                                temp = self.possible_values[x,y]
                                temp.remove(b)
                                self.possible_values[x,y] = temp 
                #check row for naked triple subset
                triples = []
                for value in self.possible_values[x]:
                    count = len(value)
                    if count == 3:
                        triples.append(value)
                if len(triples) == 3:
                    if triples[0] == triples[1] == triples[2]:
                        a = triples[0][0]
                        b = triples[0][1]
                        c = triples[0][2]
                        for y in range(9):
                            if a in self.possible_values[x,y] and self.possible_values[x,y] != triples[0]:
                                temp = self.possible_values[x,y]
                                temp.remove(a)
                                self.possible_values[x,y] = temp
                            if b in self.possible_values[x,y] and self.possible_values[x,y] != triples[0]:
                                temp = self.possible_values[x,y]
                                temp.remove(b)
                                self.possible_values[x,y] = temp
                            if c in self.possible_values[x,y] and self.possible_values[x,y] != triples[0]:
                                temp = self.possible_values[x,y]
                                temp.remove(c)
                                self.possible_values[x,y] = temp
            # check column for naked double subset
            for y in range(9):
                doubles = []
                for value in self.possible_values[:, y]:
                    count = len(value)
                    if count == 2:
                        doubles.append(value)
                if len(doubles) == 2:
                    if doubles[0] == doubles[1]:
                        a = doubles[0][0]
                        b = doubles[0][1]
                        for x in range(9):
                            if a in self.possible_values[x,y] and self.possible_values[x,y] != doubles[0]:
                                temp = self.possible_values[x,y]
                                temp.remove(a)
                                self.possible_values[x,y] = temp
                            if b in self.possible_values[x,y] and self.possible_values[x,y] != doubles[0]:
                                temp = self.possible_values[x,y]
                                temp.remove(b)
                                self.possible_values[x,y] = temp
            # check column for naked triple subset
                triples = []
                for value in self.possible_values[:, y]:
                    count = len(value)
                    if count == 3:
                        triples.append(value)
                if len(triples) == 3:
                    if triples[0] == triples[1] == triples[2]:
                        a = triples[0][0]
                        b = triples[0][1]
                        c = triples[0][2]
                        for x in range(9):
                            if a in self.possible_values[x,y] and self.possible_values[x,y] != triples[0]:
                                temp = self.possible_values[x,y]
                                temp.remove(a)
                                self.possible_values[x,y] = temp
                            if b in self.possible_values[x,y] and self.possible_values[x,y] != triples[0]:
                                temp = self.possible_values[x,y]
                                temp.remove(b)
                                self.possible_values[x,y] = temp
                            if c in self.possible_values[x,y] and self.possible_values[x,y] != triples[0]:
                                temp = self.possible_values[x,y]
                                temp.remove(c)
                                self.possible_values[x,y] = temp           
        # sole candidate - removed as made code slower as not enough sole candidates in sudokus
#             for x in range(9):
#                 for y in range(9):
#                     values = [1,2,3,4,5,6,7,8,9]
#                     for value in values:
#                         if value in self.sudoku[x] and value in values:
#                             values.remove(value)
#                         if value in self.sudoku[:, y] and value in values:
#                             values.remove(value)
#                     if len(values) == 1:
#                         print("placing sole candidate", values[0], "at ", x, y)
#                         self.possible_values[x,y] = [values[0]]
            self.strip_multiple_values()

    def check_is_invalid(self):
        """
        Checks that there is only one of each number in each row/column/square
        
        Output
            Boolean indicating if the sudoku is valid or not
        """
        # checks if all boxes have a possible value
        for lst in self.possible_values:
            for item in lst:
                if len(item) == 0:
                    return True
        if self.check_if_sudoku_solved():
            # checks if a number is repeated in rows
            for row in self.possible_values:
                for num in range(1,10):
                    count = 0
                    for x in row:
                        count += x.count(num)
                    if count > 1:
                        return True
            # checks if a number is repeated in columns
            transpose = self.possible_values.transpose()
            for column in transpose:
                for num in range(1,10):
                    count = 0
                    for x in row:
                        count += x.count(num)
                    if count > 1:
                        return True
        return False
    
    def strip_multiple_values(self):
        """
        Removes values that can be removed from multiple possible values

        """
        # adds single values to the sudoku before starting
        self.add_to_sudoku()
        for x in range(9):
            for y in range(9):
                # if sudoku started with a value then assign that as the possible value
                if self.sudoku[x,y] != 0:
                    self.possible_values[x,y] = [self.sudoku[x,y]]
                else:
                    # if it was blank to begin with then check if values can be removed
                    self.possible_values[x,y] = self.assign_possible_values(self.possible_values[x,y], x, y)

    def check_if_sudoku_solved(self):
        """
        Checks if all every list in possible value array only has one item
        
        Output
            Boolean indicating if all squares have one possible value
        """
        for row in self.possible_values:
            for item in row:
                if len(item) > 1:
                    return False
        return True
    
    def add_to_sudoku(self):
        """
        Adds single value lists to sudoku
        
        """
        for x in range(9):
            for y in range(9):
                if len(self.possible_values[x,y]) == 1:
                    self.sudoku[x,y] = self.possible_values[x,y][0]
    
    def finish_sudoku(self):
        """
        Adds all lists to sudoku
        
        """
        for x in range(9):
            for y in range(9):
                self.sudoku[x,y] = self.possible_values[x,y][0]
                
    def set_value(self, index, number):
        """
        Sets values in new state following the guessing of a number in the search
        
        Input
            List coordinates of number being guessed, int number being guessed
        
        Output
            the new state
        
        """
        # creates deep copy of old state so old state is not altered if new guess is incorrect
        state = copy.deepcopy(self)
        # guess is added to possible values
        state.possible_values[index[0], index[1]] = [number]
        for x in range(9):
            for y in range(9):
                # guess is added to sudoku
                state.add_to_sudoku()
                # possible values are altered in accordance to new value
                state.assign_possible_values(state.possible_values[x,y], x, y)
        state.constrain_values()
        return state
    

All of your code must go above this cell. You may add additional cells into the notebook if you wish, but do not duplicate or copy/paste cells as this can interfere with the grading script.

### Testing Details
There are four difficulties of sudoku provided: very easy, easy, medium, and hard. There are 15 sample sudokus in each category, with solutions as well. Difficulty was determined using reference solvers, but your code may vary; it is conceivable that your code will find some sudokus much easier or harder within a given category, or even between categories.

*All categories that are easy and above will contain* ***invalid initial states***, that is, sudoku puzzles with no solution. In this case, your function should return a 9x9 NumPy array whose values are all equal to -1.

When we test your code, we will firstly test it on the *same* very easy puzzles that you have been given. Then we will test it on additional *hidden* sudokus from each difficulty in turn, easy and up – many more than 15 of each. Grades are awarded based on whether your code can solve the puzzles. For high grades on the hard puzzles, execution time will also be a factor. 

All puzzles must take under 20 seconds each on the test machine to count as successful – if it takes longer, it will timeout. Note that this is a maximum, not a goal. Higher grades require better performance on harder puzzles. As a very rough benchmark, you should be aiming for an average of under a second per puzzle. Hardware varies, but all tests will take place on the same modern desktop machine. Our ‘standard constraint satisfaction’ implementation takes about 0.001 seconds per puzzle for the very easy category, but struggles to solve some of the hard puzzles within the time limit.

***The hard sudokus are labelled as hard for a reason.*** We expect most submissions will not be able to solve them in a reasonable length of time. Use the stop button (■) on the toolbar if you need to terminate your code because it is taking too long.

The best way to improve the performance of your code is through a detailed understanding and smart choice of AI algorithms and data structures. This assignment is ***not*** meant to test your ability to write multi-threaded code or any other kind of high-performance code optimisations. 

#### Test Cell
The following code will run your solution over the provided sudoku puzzles. To enable it, set the constant `SKIP_TESTS` to `False`. If you fail any tests of one difficulty, the code will stop, but you can modify this behaviour if you like.

**IMPORTANT**: you must set `SKIP_TESTS` back to `True` before submitting this file!

In [2]:
SKIP_TESTS = False

def tests():
    import time
    difficulties = ['very_easy', 'easy', 'medium', 'hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        for i in range(len(sudokus)):  
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print("No, the correct solution is:")
                print(solutions[i])
            
            print("This sudoku took", end_time-start_time, "seconds to solve.\n")

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        if count < len(sudokus):
            break
            
if not SKIP_TESTS:
    tests()
    

Testing very_easy sudokus
This is very_easy sudoku number 0
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]]
This is your solution for very_easy sudoku number 0
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.0 seconds to solve.

This is very_easy sudoku number 1
[[0 9 3 1 5 2 6 0 8]
 [8 6 2 7 0 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 0 3 6]
 [5 0 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]
 [6 1 4 2 7 8 9 5 3]
 [7 3 9 6 1 5 8 4 2]
 [2 8 5 3 9 4 7 6 1]]
This is your solution for very_easy sudoku number 1
[[4 9 3 1 5 2 6 7 8]
 [8 6 2 7 4 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 5 3 6]
 [5 2 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]
 [6 1 4 2 7 8 9 5

This sudoku took 0.0 seconds to solve.

This is easy sudoku number 10
[[8 2 6 9 3 4 1 5 7]
 [9 5 3 7 1 8 0 4 0]
 [7 4 0 2 6 5 3 9 8]
 [6 9 2 4 8 7 5 1 3]
 [1 7 5 3 2 6 9 8 4]
 [4 3 8 5 9 1 2 7 6]
 [2 8 9 1 4 0 7 6 5]
 [5 1 4 6 7 2 8 3 9]
 [3 6 7 8 5 9 4 0 1]]
This is your solution for easy sudoku number 10
[[8 2 6 9 3 4 1 5 7]
 [9 5 3 7 1 8 6 4 2]
 [7 4 1 2 6 5 3 9 8]
 [6 9 2 4 8 7 5 1 3]
 [1 7 5 3 2 6 9 8 4]
 [4 3 8 5 9 1 2 7 6]
 [2 8 9 1 4 3 7 6 5]
 [5 1 4 6 7 2 8 3 9]
 [3 6 7 8 5 9 4 2 1]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.0 seconds to solve.

This is easy sudoku number 11
[[3 5 4 9 2 6 8 1 7]
 [7 2 6 1 3 8 5 9 4]
 [8 9 1 4 0 7 2 0 3]
 [1 8 5 2 9 4 3 7 6]
 [4 6 3 7 1 5 9 2 8]
 [9 7 2 6 8 3 1 4 5]
 [5 4 9 3 7 1 6 0 2]
 [6 1 8 0 4 2 7 0 9]
 [2 3 7 8 6 9 4 5 1]]
This is your solution for easy sudoku number 11
[[3 5 4 9 2 6 8 1 7]
 [7 2 6 1 3 8 5 9 4]
 [8 9 1 4 5 7 2 6 3]
 [1 8 5 2 9 4 3 7 6]
 [4 6 3 7 1 5 9 2 8]
 [9 7 2 6 8 3 1 4 5]
 [5 4 9 3 7 1 6 8 2

This is your solution for medium sudoku number 12
[[5 4 6 7 8 9 3 1 2]
 [8 3 1 2 5 6 9 4 7]
 [2 9 7 3 4 1 8 6 5]
 [3 7 5 6 2 8 1 9 4]
 [9 6 2 4 1 3 5 7 8]
 [4 1 8 5 9 7 6 2 3]
 [1 2 9 8 3 4 7 5 6]
 [7 5 3 9 6 2 4 8 1]
 [6 8 4 1 7 5 2 3 9]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.015625 seconds to solve.

This is medium sudoku number 13
[[0 7 8 0 5 2 4 0 3]
 [5 1 9 0 7 0 8 6 2]
 [0 4 3 8 0 6 5 7 1]
 [1 3 6 9 2 4 7 0 8]
 [0 0 0 5 3 1 9 0 6]
 [9 2 0 7 0 8 1 3 4]
 [0 6 7 0 4 5 2 1 9]
 [3 0 1 2 8 7 6 4 5]
 [4 0 2 0 1 9 3 8 0]]
This is your solution for medium sudoku number 13
[[6 7 8 1 5 2 4 9 3]
 [5 1 9 4 7 3 8 6 2]
 [2 4 3 8 9 6 5 7 1]
 [1 3 6 9 2 4 7 5 8]
 [7 8 4 5 3 1 9 2 6]
 [9 2 5 7 6 8 1 3 4]
 [8 6 7 3 4 5 2 1 9]
 [3 9 1 2 8 7 6 4 5]
 [4 5 2 6 1 9 3 8 7]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.0 seconds to solve.

This is medium sudoku number 14
[[3 2 1 8 5 4 0 9 7]
 [5 7 4 0 0 6 8 2 1]
 [6 0 8 1 2 0 3 0 4]
 [8 3 0 0 0 9 1 4 2]

This is your solution for hard sudoku number 12
[[-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 1.625 seconds to solve.

This is hard sudoku number 13
[[0 0 2 0 0 3 0 7 1]
 [0 0 0 0 0 6 0 0 0]
 [0 1 5 0 0 0 0 8 0]
 [0 0 0 7 0 0 0 0 8]
 [1 0 0 0 0 0 0 0 6]
 [0 2 4 0 1 5 0 0 0]
 [0 0 0 0 2 0 0 0 3]
 [0 7 0 1 0 0 0 6 0]
 [0 9 0 0 5 0 0 0 0]]
This is your solution for hard sudoku number 13
[[-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]]
Is your solution correct?
Yes! Correct solution.
Th

## Submission Test
The following cell tests if your notebook is ready for submission. **You must not skip this step!**

Restart the kernel and run the entire notebook (Kernel → Restart & Run All). Now look at the output of the cell below. 

*If there is no output, then your submission is not ready.* Either your code is still running (did you forget to skip tests?) or it caused an error.

As previously mentioned, failing to follow these instructions can result in a grade of zero.

In [4]:
def submission_tests():
    import sys
    import pathlib

    fail = False;

    if not SKIP_TESTS:
        fail = True;
        print("You must set the SKIP_TESTS constant to True in the cell above.")

    p1 = pathlib.Path('./readme.txt')
    p2 = pathlib.Path('./readme.md')
    if not (p1.is_file() or p2.is_file()):
        fail = True;
        print("You must include a separate file called readme.txt or readme.md in your submission.")

    p3 = pathlib.Path('./sudoku.ipynb')
    if not p3.is_file():
        fail = True
        print("This notebook file must be named sudoku.ipynb")

    if "sudoku_solver" not in globals():
        fail = True;
        print("You must include a function called sudoku_solver which accepts a numpy array.")
    else: 
        sudoku = np.load("data/very_easy_puzzle.npy")[0]
        solution = np.load("data/very_easy_solution.npy")[0]

        if not np.array_equal(sudoku_solver(sudoku), solution):
            print("Warning:")
            print("Your sudoku_solver function does not correctly solve the first sudoku.")
            print()
            print("Your assignment is unlikely to get any marks from the autograder. While we will")
            print("try to check it manually to assign some partial credit, we encourage you to ask")
            print("for help on the forum or directly to a tutor.")
            print()
            print("Please use the readme file to explain your code anyway.")

    if fail:
        print()
        sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
    else:
        print("All checks passed. When you are ready to submit, upload the notebook and readme file to the")
        print("assignment page, without changing any filenames.")
        print()
        print("If you need to submit multiple files, you can archive them in a .zip file. (No other format.)")
        
submission_tests()

All checks passed. When you are ready to submit, upload the notebook and readme file to the
assignment page, without changing any filenames.

If you need to submit multiple files, you can archive them in a .zip file. (No other format.)


In [5]:
# This is a TEST CELL. Do not delete or change.